# Service Creation
## Load Config

In [92]:
import json
import os

In [93]:
PROJECT_PATH = "/Users/vinay/Projects/my-mlops/"

In [94]:
def load_app_config():
    config_path = os.path.join(PROJECT_PATH, "config", "config.json")
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Config file not found: {config_path}")
    with open(config_path, "r") as f:
        return json.load(f)

cfg = load_app_config()

## Resource Group

In [98]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient

In [99]:
# loading subscription id
with open(os.path.join(PROJECT_PATH,"tokens","subscription_id"), "r") as f:
    SUBSCRIPTION_ID = f.read()

In [100]:
RG_CONFIG = cfg.get("resource_group", {})
RESOURCE_GROUP = RG_CONFIG.get("name")
LOCATION = RG_CONFIG.get("location", "eastus")

In [101]:
def get_resource_client(subscription_id):
    """Authenticate and return the ResourceManagementClient"""
    credential = DefaultAzureCredential()
    return ResourceManagementClient(credential, subscription_id)

In [102]:
resource_client = get_resource_client(SUBSCRIPTION_ID)

In [103]:
def validate_resource_group(resource_client, RESOURCE_GROUP):
    """Check if a resource group exists"""
    print(f"🔍 Validating resource group '{RESOURCE_GROUP}'...")
    exists = resource_client.resource_groups.check_existence(RESOURCE_GROUP)
    if exists:
        print(f"✅ Resource group '{RESOURCE_GROUP}' exists.")
    else:
        print(f"❌ Resource group '{RESOURCE_GROUP}' does not exist.")
    return exists

### Validating if resource group is available if not create one

In [104]:
if validate_resource_group(resource_client, RESOURCE_GROUP):
    print(f"ℹ️ Resource group '{RESOURCE_GROUP}' already exists.")
else:
    print(f"🚀 Creating resource group '{RESOURCE_GROUP}' in '{LOCATION}'...")
    rg_result = resource_client.resource_groups.create_or_update(
        RESOURCE_GROUP, {"location": LOCATION}
    )
    print(f"✅ Resource group '{rg_result.name}' created successfully.")

🔍 Validating resource group 'rg-mymlops'...
✅ Resource group 'rg-mymlops' exists.
ℹ️ Resource group 'rg-mymlops' already exists.


## Virtual machine

In [105]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.compute import ComputeManagementClient
from azure.mgmt.network import NetworkManagementClient
from azure.mgmt.compute.models import (
    HardwareProfile, OSProfile, LinuxConfiguration,
    SshConfiguration, SshPublicKey, NetworkInterfaceReference,
    StorageProfile, ImageReference, OSDisk, DiskCreateOptionTypes
)
from azure.mgmt.network.models import (
    NetworkSecurityGroup, SecurityRule, PublicIPAddress,
    NetworkInterface, Subnet, VirtualNetwork, IPConfiguration
)

In [106]:
VM_NAME = cfg["vm"]["name"]
VM_SIZE = cfg["vm"]["size"]
ADMIN_USERNAME = cfg["vm"]["admin_username"]
SSH_KEY_PATH = os.path.expanduser(cfg["vm"]["ssh_public_key_path"])
VIRTUAL_NETWORK = cfg["vm"]["virtual_network"]
SUB_NETWORK = cfg["vm"]["sub_network"]
NETWORK_SECURITY_GRP = cfg["vm"]["network_security_grp"]
PUBLIC_IP = cfg["vm"]["public_ip"]
NETWORK_INTERFACE = cfg["vm"]["network_interface"]

In [107]:
print(f"🚀 Starting VM creation: {VM_NAME} in {LOCATION}")

credential = DefaultAzureCredential()
resource_client = ResourceManagementClient(credential, SUBSCRIPTION_ID)
network_client = NetworkManagementClient(credential, SUBSCRIPTION_ID)
compute_client = ComputeManagementClient(credential, SUBSCRIPTION_ID)

🚀 Starting VM creation: vm-mlops in eastus


In [108]:
# Virtual Network
print("🔹 Creating Virtual Network...")
vnet = network_client.virtual_networks.begin_create_or_update(
    RESOURCE_GROUP, VIRTUAL_NETWORK,
    {
        "location": LOCATION, 
        "address_space": {"address_prefixes": ["10.0.0.0/16"]}
    },
).result()
print(f"✅ Virtual Network '{vnet.name}' created successfully.")

🔹 Creating Virtual Network...
✅ Virtual Network 'vnet-mlops' created successfully.


In [ ]:
# Subnet
print("🔹 Creating Subnet...")
subnet = network_client.subnets.begin_create_or_update(
    RESOURCE_GROUP, VIRTUAL_NETWORK, 
    SUB_NETWORK, {"address_prefix": "10.0.0.0/24"},
).result()
print(f"✅ Subnet '{subnet.name}' created successfully.")

🔹 Creating Subnet...
✅ Subnet 'subnet-mlops' created successfully.


In [ ]:
# NSG (SSH only)
print("🔹 Creating NSG (allow SSH)...")
nsg = network_client.network_security_groups.begin_create_or_update(
    RESOURCE_GROUP, NETWORK_SECURITY_GRP,
    {
        "location": LOCATION,
        "security_rules": [
            SecurityRule(
                name="AllowSSH", protocol="Tcp", direction="Inbound", 
                access="Allow", priority=1000, source_address_prefix="*", 
                destination_address_prefix="*", source_port_range="*", 
                destination_port_range="22",
            )
        ],
    },
).result()
print(f"✅ NSG '{nsg.name}' created successfully.")

🔹 Creating NSG (allow SSH)...
✅ NSG 'nsg-mlops' created successfully.


In [ ]:
# Public IP
print("🔹 Creating Public IP...")
public_ip = network_client.public_ip_addresses.begin_create_or_update(
    RESOURCE_GROUP, PUBLIC_IP, 
    {"location": LOCATION, "public_ip_allocation_method":"Dynamic"},
    ).result()
print(f"✅ NSG '{public_ip.name}' created successfully.")
# print(f"✅ NSG '{public_ip.ip_address}' created successfully.")

🔹 Creating Public IP...
✅ NSG 'pip-mlops' created successfully.


In [ ]:
# Network Interface
print("🔹 Creating Network Interface...")
nic = network_client.network_interfaces.begin_create_or_update(
    RESOURCE_GROUP, NETWORK_INTERFACE,
    {
        "location": LOCATION,
        "ip_configurations": [
            IPConfiguration(
                name="ipconfig1",
                subnet=Subnet(id=subnet.id),
                public_ip_address=PublicIPAddress(id=public_ip.id),
                network_security_group=NetworkSecurityGroup(id=nsg.id),
            )
        ],
    },
).result()
print(f"✅ NSG '{nic.name}' created successfully.")

network_security_group is not a known attribute of class <class 'azure.mgmt.network.models._models_py3.IPConfiguration'> and will be ignored


🔹 Creating Network Interface...
✅ NSG 'nic-mlops' created successfully.


In [ ]:
# VM Configuration
print("🔹 Defining VM configuration...")
with open(SSH_KEY_PATH, "r") as f:
    ssh_key_data = f.read().strip()

vm_parameters = {
    "location": LOCATION,
    "hardware_profile": HardwareProfile(vm_size=VM_SIZE),
    "storage_profile": StorageProfile(
        image_reference=ImageReference(
            publisher="Canonical",
            offer="UbuntuServer",
            sku="18.04-LTS",
            version="latest",
        ),
        os_disk=OSDisk(
            create_option=DiskCreateOptionTypes.from_image,
            managed_disk={"storage_account_type": "StandardSSD_LRS"},
        ),
    ),
    "os_profile": OSProfile(
        computer_name=VM_NAME,
        admin_username=ADMIN_USERNAME,
        linux_configuration=LinuxConfiguration(
            disable_password_authentication=True,
            ssh=SshConfiguration(
                public_keys=[
                    SshPublicKey(
                        path=f"/home/{ADMIN_USERNAME}/.ssh/authorized_keys",
                        key_data=ssh_key_data,
                    )
                ]
            ),
        ),
    ),
    "network_profile": {
        "network_interfaces": [NetworkInterfaceReference(id=nic.id)]},
}

🔹 Defining VM configuration...


In [ ]:
# Create the VM
print(f"🚀 Creating VM '{VM_NAME}' ... this may take a few minutes.")
creation = compute_client.virtual_machines.begin_create_or_update(
    RESOURCE_GROUP, VM_NAME, vm_parameters
)
vm_result = creation.result()

🚀 Creating VM 'vm-mlops' ... this may take a few minutes.


In [109]:
# Fetch VM details
vm_details = compute_client.virtual_machines.get(RESOURCE_GROUP, VM_NAME)
# Print summary
print("\n🧾 VM Details:")
print(f"Name: {vm_details.name}")
print(f"Location: {vm_details.location}")
print(f"Size: {vm_details.hardware_profile.vm_size}")
print(f"OS Disk: {vm_details.storage_profile.os_disk.name}")
print(f"Image: {vm_details.storage_profile.image_reference.offer} {vm_details.storage_profile.image_reference.sku}")
print(f"Provisioning State: {vm_details.provisioning_state}")

ResourceNotFoundError: (ResourceNotFound) The Resource 'Microsoft.Compute/virtualMachines/vm-mlops' under resource group 'rg-mymlops' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix
Code: ResourceNotFound
Message: The Resource 'Microsoft.Compute/virtualMachines/vm-mlops' under resource group 'rg-mymlops' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix

In [ ]:
# Get NIC attached to the VM
nic_id = vm_details.network_profile.network_interfaces[0].id.split('/')[-1]

# Fetch NIC details
nic = network_client.network_interfaces.get(RESOURCE_GROUP, nic_id)

# Check if NSG is attached directly to NIC
if nic.network_security_group:
    nsg_id = nic.network_security_group.id.split('/')[-1]
    nsg = network_client.network_security_groups.get(RESOURCE_GROUP, nsg_id)
    print(f"🔒 NSG attached to NIC '{nic.name}': {nsg.name}")
    print(f"Location: {nsg.location}")
    print(f"Security Rules:")
    for rule in nsg.security_rules:
        print(f"  - {rule.name}: {rule.direction} {rule.access} {rule.protocol} {rule.destination_port_range}")
else:
    print(f"No NSG attached directly to NIC '{nic.name}'.")

🔒 NSG attached to NIC 'nic-mlops': nsg-mlops
Location: eastus
Security Rules:
  - AllowSSH: Inbound Allow Tcp 22
No NSG attached to Subnet 'subnet-mlops'.


In [28]:
# Attach NSG to NIC
nic.network_security_group = nsg

In [ ]:
# Update NIC with the NSG association
update_nic = network_client.network_interfaces.begin_create_or_update(
    RESOURCE_GROUP,
    nic_id,
    nic
)

In [31]:
result = update_nic.result()
print(f"✅ NSG '{nsg.name}' successfully attached to NIC '{nic_id}'.")

✅ NSG 'nsg-mlops' successfully attached to NIC 'nic-mlops'.


In [ ]:
if nic.network_security_group:
    nsg_id = nic.network_security_group.id.split('/')[-1]
    nsg = network_client.network_security_groups.get(RESOURCE_GROUP, nsg_id)
    print(f"🔒 NSG attached to NIC '{nic.name}': {nsg.name}")
    print(f"Location: {nsg.location}")
    print(f"Security Rules:")
    for rule in nsg.security_rules:
        print(f"  - {rule.name}: {rule.direction} {rule.access} {rule.protocol} {rule.destination_port_range}")
else:
    print(f"No NSG attached directly to NIC '{nic.name}'.")

🔒 NSG attached to NIC 'nic-mlops': nsg-mlops
Location: eastus
Security Rules:
  - AllowSSH: Inbound Allow Tcp 22


## Blob Storage

In [110]:
from azure.mgmt.storage import StorageManagementClient

In [111]:
credential = DefaultAzureCredential()
storage_client = StorageManagementClient(credential, SUBSCRIPTION_ID)

In [112]:
credential = DefaultAzureCredential()
storage_client = StorageManagementClient(credential, SUBSCRIPTION_ID)

In [116]:
STORAGE_ACCOUNT_NAME = cfg["storage"]["storage_account"]
CONTAINERS = cfg["storage"]["containers"]

In [ ]:
# Create Blob Storage with hierarchical namespace (Data Lake Gen2)
storage_async_op = storage_client.storage_accounts.begin_create(
    RESOURCE_GROUP,
    STORAGE_ACCOUNT_NAME,
    {
        "location": LOCATION,
        "kind": "StorageV2",
        "sku": {"name": "Standard_LRS"},
        "is_hns_enabled": True  # Hierarchical namespace enabled
    },
)
storage_account = storage_async_op.result()
print(f"✅ Storage account '{storage_account.name}' created.")

✅ Storage account 'storagemymlops' created.


In [ ]:
# ==== Get Storage Account Key ====
print("🔑 Retrieving access key...")
keys = storage_client.storage_accounts.list_keys(RESOURCE_GROUP, STORAGE_ACCOUNT_NAME)
storage_key = keys.keys[0].value

🔑 Retrieving access key...


ResourceNotFoundError: (ResourceNotFound) The Resource 'Microsoft.Storage/storageAccounts/storageguestsense' under resource group 'rg-mymlops' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix
Code: ResourceNotFound
Message: The Resource 'Microsoft.Storage/storageAccounts/storageguestsense' under resource group 'rg-mymlops' was not found. For more details please go to https://aka.ms/ARMResourceNotFoundFix

In [ ]:
# ==== Save Key to File ====
key_file_path = os.path.join(PROJECT_PATH, "tokens", STORAGE_ACCOUNT_NAME)
print(key_file_path)
with open(key_file_path, "w") as key_file:
    key_file.write(storage_key)
print(f"🗝️ Storage account key saved to: {key_file_path}")

/Users/vinay/Projects/my-mlops/tokens/storagemymlops
🗝️ Storage account key saved to: /Users/vinay/Projects/my-mlops/tokens/storagemymlops


In [ ]:
# ==== Get Storage Account Key ====
print("🔑 Retrieving access key...")
keys = storage_client.storage_accounts.list_keys(RESOURCE_GROUP, STORAGE_ACCOUNT_NAME)
storage_key = keys.keys[0].value

🔑 Retrieving access key...


NameError: name 'RESOURCE_GROUP' is not defined

In [ ]:
# ==== Connect to Blob Service ====
from azure.storage.blob import BlobServiceClient
blob_service_client = BlobServiceClient(
    f"https://{STORAGE_ACCOUNT_NAME}.blob.core.windows.net",
    credential=storage_key)

In [ ]:
# ==== Create Containers ====
for container in CONTAINERS.split(','):
    try:
        blob_service_client.create_container(container)
        print(f"✅ Container '{container}' created successfully.")
    except Exception as e:
        if "ContainerAlreadyExists" in str(e):
            print(f"⚠️ Container '{container}' already exists.")
        else:
            print(f"❌ Failed to create container '{container}': {e}")

⚠️ Container 'bronze' already exists.
⚠️ Container 'silver' already exists.
⚠️ Container 'gold' already exists.


### Uploading find to S3